# Cadre de validation de la qualité des données (6 piliers)

Ce notebook met en œuvre une suite complète de validations avec **Great Expectations v0.18+** sur les fichiers CSV de santé.
Toutes les cellules, commentaires et messages sont en français.

**Piliers couverts :** complétude, exactitude, validité, cohérence, unicité, actualité.

**Sorties :** résultats dans `./gx/` et rapport HTML dans `./reports/`.

In [174]:
# -*- coding: utf-8 -*-
"""
Validation Qualité des Données – Great Expectations v1.11.3
Compatible avec l'API moderne (v1.x)
"""

import os
import pandas as pd
import great_expectations as gx
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# Chemins
DATA_DIR = "./data"
GX_DIR = "./gx"
REPORTS_DIR = "./reports"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(GX_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)

print("✅ Dossiers créés.")

✅ Dossiers créés.


In [175]:
# Créer un contexte persistant dans ./gx
context = gx.get_context()

# Configurer un site Data Docs avec un chemin ABSOLU
context.update_data_docs_site(
    site_name="local_site",
    site_config={
        "class_name": "SiteBuilder",
        "store_backend": {
            "class_name": "TupleFilesystemStoreBackend",
            "base_directory": os.path.abspath(os.path.join(REPORTS_DIR, "gx_data_docs")),
        },
        "site_index_builder": {"class_name": "DefaultSiteIndexBuilder"},
    }
)

print("✅ Contexte GX initialisé (v1.11.3).")

✅ Contexte GX initialisé (v1.11.3).


In [176]:
def load_csv(filename):
    path = os.path.join(DATA_DIR, filename)
    if not os.path.exists(path):
        print(f"⚠️  {filename} manquant")
        return pd.DataFrame()
    try:
        df = pd.read_csv(path, dtype=str, na_filter=False)
        
        # 🔹 Conversion des dates pour patients.csv
        if filename == "patients.csv":
            # Convertir les colonnes de date en datetime
            for col in ["arrival_date", "departure_date", "date_naissance"]:
                if col in df.columns:
                    df[col] = pd.to_datetime(df[col], errors="coerce")
        
        # 🔹 Conversion pour consultations.csv (si besoin)
        elif filename == "consultations.csv":
            if "consultation_date" in df.columns:
                df["consultation_date"] = pd.to_datetime(df["consultation_date"], errors="coerce")
        
        # 🔹 (Optionnel) Conversion pour staff.csv
        elif filename == "staff.csv":
            if "date_naissance" in df.columns:
                df["date_naissance"] = pd.to_datetime(df["date_naissance"], errors="coerce")

        return df
    except Exception as e:
        print(f"❌ Erreur chargement {filename}: {e}")
        return pd.DataFrame()

# Charger les fichiers
staff_df = load_csv("staff.csv")
patients_df = load_csv("patients.csv")
consultations_df = load_csv("consultations.csv")
schedule_df = load_csv("staff_schedule.csv")
services_df = load_csv("services_weekly.csv")

In [177]:
# Sauvegarder les DataFrames localement pour GX
staff_df.to_csv(os.path.join(GX_DIR, "staff.csv"), index=False)
patients_df.to_csv(os.path.join(GX_DIR, "patients.csv"), index=False)
consultations_df.to_csv(os.path.join(GX_DIR, "consultations.csv"), index=False)
schedule_df.to_csv(os.path.join(GX_DIR, "schedule.csv"), index=False)
services_df.to_csv(os.path.join(GX_DIR, "services.csv"), index=False)

# Ajouter une source de données locale (API v3)
datasource = context.data_sources.add_pandas(name="local_files")

# Créer des Data Assets
staff_asset = datasource.add_csv_asset(
    name="staff",
    filepath_or_buffer=os.path.join(GX_DIR, "staff.csv")
)

patients_asset = datasource.add_csv_asset(
    name="patients",
    filepath_or_buffer=os.path.join(GX_DIR, "patients.csv")
)

consultations_asset = datasource.add_csv_asset(
    name="consultations",
    filepath_or_buffer=os.path.join(GX_DIR, "consultations.csv")
)

schedule_asset = datasource.add_csv_asset(
    name="schedule",
    filepath_or_buffer=os.path.join(GX_DIR, "schedule.csv")
)

services_asset = datasource.add_csv_asset(
    name="services",
    filepath_or_buffer=os.path.join(GX_DIR, "services.csv")
)

print("✅ Data Assets créés.")


✅ Data Assets créés.


In [203]:
# Réinitialiser results
results = []

# Liste des combinaisons (asset, suite) à valider
validations_to_run = [
    ("staff", "completude_staff"),
    ("patients", "completude_patients"),
    ("consultations", "completude_consultations"),
    ("staff", "exactitude_staff"),
    ("patients", "exactitude_patients"),
    ("schedule", "exactitude_schedule"),
    ("staff", "validite_staff"),
    ("patients", "validite_patients"),
    ("services", "validite_services"),
    ("staff", "unicite_staff"),
    ("patients", "unicite_patients"),
    ("schedule", "unicite_schedule"),
    ("patients", "actualite_patients"),  # ✅ Décommenté !
]

for asset_name, suite_name in validations_to_run:
    df = assets[asset_name]
    if df.empty:
        continue
        
    try:
        # Récupérer la suite existante
        suite = context.suites.get(suite_name)
        
        # Créer le batch request
        batch_request = locals()[f"{asset_name}_asset"].build_batch_request()
        
        # Créer le validator
        validator = context.get_validator(
            batch_request=batch_request,
            expectation_suite=suite
        )
        
        # Exécuter la validation
        result = validator.validate()
        success = result.success
        
        # Extraire le pilier depuis le nom de la suite
        pilier = suite_name.split("_")[0]
        
        results.append({
            "Pilier": pilier,
            "Dataset": asset_name,
            "Succès": success
        })
        
        print(f"✅ Validation réussie : {asset_name} / {pilier}")
        
    except Exception as e:
        print(f"⚠️  Erreur validation {asset_name}/{suite_name}: {e}")

# Générer le rapport HTML
context.build_data_docs()
html_path = os.path.abspath(os.path.join(REPORTS_DIR, "gx_data_docs/index.html"))
print(f"\n📄 Rapport HTML généré : {html_path}")

# Ouvrir dans le navigateur
import webbrowser
webbrowser.open(f"file://{html_path}")

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

✅ Validation réussie : staff / completude


Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

✅ Validation réussie : patients / completude


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

✅ Validation réussie : patients / unicite


Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Validation réussie : schedule / unicite
⚠️  Erreur validation patients/actualite_patients: ExpectationSuite with name actualite_patients was not found.

📄 Rapport HTML généré : c:\Users\ababid\Desktop\data gouv\reports\gx_data_docs\index.html


True

In [204]:
print("\n🔍 Premier élément de results :", results[0] if results else "vide")
df_results = pd.DataFrame(results)
print("Colonnes du DataFrame :", df_results.columns.tolist())


🔍 Premier élément de results : {'Pilier': 'completude', 'Dataset': 'staff', 'Succès': True}
Colonnes du DataFrame : ['Pilier', 'Dataset', 'Succès']


In [205]:
if results:
    print("Clés dans le premier résultat :", list(results[0].keys()))
    df_results = pd.DataFrame(results)
    print("Colonnes du DataFrame :", df_results.columns.tolist())
else:
    print("⚠️  Aucune validation exécutée.")

if results:
    df_results = pd.DataFrame(results)
    
    # Calcul des scores moyens par pilier
    scores = df_results.groupby("Pilier")["Succès"].mean().sort_values(ascending=False) * 100
    
    print("\n📊 SYNTHÈSE DES PILIERS DE QUALITÉ")
    print("=" * 50)
    for pilier, score in scores.items():
        # Barre proportionnelle (max 10 blocs)
        bar_length = min(int(score // 10), 10)
        bar = "█" * bar_length + "░" * (10 - bar_length)
        print(f"{pilier:12} | {score:5.1f}% | {bar}")
    
    # Score global
    score_global = df_results["Succès"].mean() * 100
    print("-" * 50)
    print(f"{'GLOBAL':12} | {score_global:5.1f}% | {'='*10}")
    
else:
    print("⚠️  Aucune validation exécutée.")

print("\n✅ Validation terminée. Rapport HTML disponible dans ./reports/")

Clés dans le premier résultat : ['Pilier', 'Dataset', 'Succès']
Colonnes du DataFrame : ['Pilier', 'Dataset', 'Succès']

📊 SYNTHÈSE DES PILIERS DE QUALITÉ
completude   |  66.7% | ██████░░░░
validite     |  66.7% | ██████░░░░
exactitude   |  33.3% | ███░░░░░░░
unicite      |  33.3% | ███░░░░░░░
--------------------------------------------------
GLOBAL       |  50.0% | ==========

✅ Validation terminée. Rapport HTML disponible dans ./reports/
